In [160]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

In [161]:
train_data = pd.read_csv("train.csv", index_col='id')
test_data = pd.read_csv("test.csv")
train_data.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [162]:
train_data.shape, test_data.shape

((188533, 12), (125690, 12))

In [163]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [164]:
def features(df):
    
    segment_mapping = {
    'Luxury & Premium': [
        'Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Lexus', 'Cadillac', 
        'Jaguar', 'Land', 'Tesla', 'INFINITI', 'Lincoln', 'Acura', 
        'Genesis', 'Volvo', 'Alfa', 'Maserati', 'Lucid'
    ],
    'Ultra-Luxury & Exotic': [
        'Rolls-Royce', 'Bentley', 'Ferrari', 'Lamborghini', 'McLaren', 
        'Aston', 'Bugatti', 'Maybach'
    ],
    'Mainstream': [
        'Toyota', 'Honda', 'Nissan', 'Ford', 'Chevrolet', 'Hyundai', 
        'Kia', 'Mazda', 'Subaru', 'Volkswagen', 'Buick', 'Chrysler', 
        'Dodge', 'GMC', 'RAM', 'Jeep'
    ],
    'Budget & Economy': [
        'Mitsubishi', 'FIAT', 'smart', 'Suzuki', 'MINI'
    ],
    'Defunct, Niche & Special': [
        'Pontiac', 'Hummer', 'Saturn', 'Scion', 'Mercury', 'Saab', 
        'Plymouth', 'Karma', 'Lotus', 'Polestar', 'Rivian'
    ]
    }
    def seg_brand(brand_col):
        for segment, brandlist in segment_mapping.items():
            if brand_col in brandlist:
                return segment
        return "Unknown"
    df['brand_segmented'] = df['brand'].apply(seg_brand)
    
    neutral_colors = ['black', 'white', 'grey', 'silver']
    bright_colors = ['red', 'blue', 'yellow', 'green', 'orange']
    
    
    def categorize_color(color):
        color = color.lower()
        if any(neutral in color for neutral in neutral_colors):
            return 'Neutral'
        elif any(bright in color for bright in bright_colors):
            return 'Bright'
        else:
            return 'Special'
    df['ext_col_category']= df['ext_col'].apply(categorize_color)
    
    df['age_car'] = 2024 - df['model_year']
    df.loc[df.age_car == 0, 'age_car'] = 1
    
    df['transmission_type'] = "Unknown"
    df.loc[df['transmission'].str.contains('A/T|Automatic|AT|Dual Shift Mode|CVT|AT', na=False, case=False), 'transmission_type'] = 'Automatic'
    df.loc[df['transmission'].str.contains('M/T|Manual|MT', na=False, case=False), 'transmission_type'] = 'Manual'
    df.loc[df['transmission'].str.contains('at/mt|mt/at', na=False, case=False), 'transmission_type'] = 'Manual'
    
    df.loc[df['engine'].str.contains('Electric|Battery|kW|charge|kw', na=False, case=False), 'fuel_type'] = "Electric"
    
    df.loc[(df['fuel_type'].isna()) & df['engine'].str.contains('Hybrid', na=False, case=False), 'fuel_type'] = "Hybrid"
    df.loc[(df['fuel_type'].isna()) & df['engine'].str.contains('Diesel|diesel'), 'fuel_type'] = 'Diesel'
    df.loc[(df['fuel_type'].isna()) & df['engine'].str.contains('flex|Flex'), 'fuel_type'] = 'E85 Flex Fuel'
    df.loc[(df['fuel_type'].isna()) & df['engine'].str.contains(r'\bElectric\b|\bV\b', regex=True), 'fuel_type'] = 'Electric'
    df.loc[(df['fuel_type'].isna()) & df['engine'].str.contains('Gasoline|Standard|Turbo|Liter|GDI|MPFI|PDI'), 'fuel_type'] = 'Gasoline'
    df['fuel_type'].fillna(df['fuel_type'].mode()[0], inplace=True)
    
    df['horsePower'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
    
    df['displacement'] = df['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
    
    df['Cylinder'] = df['engine'].str.extract(r'(\d*) Cylinder').astype(float)
    
    df['horsePower'] = round(df['horsePower'].fillna(df['horsePower'].mean()),1)
    
    df['displacement'] = round(df['displacement'].fillna(df['displacement'].mean()),1)
    
    df['Cylinder'] = round(df['Cylinder'].fillna(df['Cylinder'].mean()),1)
    
    df.loc[(df.accident.isna()) & (df.milage < 50000), 'accident'] = 'None reported'
    df['accident'] = df['accident'].fillna('At least 1 accident or damage reported')
    df.loc[df.accident == 'None reported', 'clean_title'] = 'Yes'
    df.loc[(df.clean_title.isna()) & (df.accident == 'At least 1 accident or damage reported'), 'clean_title'] = 'No'
    df['clean_title'] = df['clean_title'].fillna("Yes")
    df[df['clean_title']==1] = 'Yes'
    
    df['milage_age'] = df['milage'].astype(float) * df['age_car'].astype(float)
    
    df['milage_age_ratio'] = df['milage'] / df['age_car']
    
    df['mean_milage_with_age'] = df.groupby(['age_car'])['milage'].transform('mean')
    
    df['mean_milage_age_ratio_with_age'] = df.groupby(['age_car'])['milage_age_ratio'].transform('mean')
    
    df['engine_efficiency_1'] = df['horsePower'] / df['displacement']
    
    df['engine_efficiency_2'] = df['horsePower'] / df['Cylinder']
    
    df['displacement_per_cylinder'] = df['displacement'] / df['Cylinder']

    # numerical_features = df.drop(['price'], axis=1).select_dtypes(exclude = ['object', 'category']).columns
    # categorical_features = df.drop(['price'], axis=1).select_dtypes(include = ['object', 'category']).columns
    
    return df.select_dtypes(exclude = ['object', 'category'])

In [165]:
train_data = features(train_data)
test_data = features(test_data)

C:\Users\muham\AppData\Local\Temp\ipykernel_6728\2040438606.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fuel_type'].fillna(df['fuel_type'].mode()[0], inplace=True)
C:\Users\muham\AppData\Local\Temp\ipykernel_6728\2040438606.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a